<a href="https://colab.research.google.com/github/srivatsan88/Mastering-Apache-Spark/blob/master/Apache_Spark_3_0_ML_on_GPU_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Aug 27 04:23:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.14/cudf-0.14-cuda10-1.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.0.0-0.1.0/xgboost4j-spark_3.0-1.0.0-0.1.0.jar
!wget http://insecure.repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/0.1.0/rapids-4-spark_2.12-0.1.0.jar

In [ ]:
!ls
!pwd

covtype_test.parquet		 sample_data
covtype_test.parquet.1		 spark-3.0.0-bin-hadoop2.7
covtype_test.parquet.2		 spark-3.0.0-bin-hadoop2.7.tgz
covtype_train.parquet		 spark-3.0.0-bin-hadoop2.7.tgz.1
covtype_train.parquet.1		 spark-3.0.0-bin-hadoop2.7.tgz.2
covtype_train.parquet.2		 spark-3.0.0-bin-hadoop2.7.tgz.3
cudf-0.14-cuda10-1.jar		 spark-3.0.0-bin-hadoop2.7.tgz.4
cudf-0.14-cuda10-1.jar.1	 xgboost4j_3.0-1.0.0-0.1.0.jar
cudf-0.14-cuda10-1.jar.2	 xgboost4j_3.0-1.0.0-0.1.0.jar.1
cudf-0.14-cuda10-1.jar.3	 xgboost4j_3.0-1.0.0-0.1.0.jar.2
cudf-0.14-cuda10-1.jar.4	 xgboost4j_3.0-1.0.0-0.1.0.jar.3
model				 xgboost4j_3.0-1.0.0-0.1.0.jar.4
rapids-4-spark_2.12-0.1.0.jar	 xgboost4j-spark_3.0-1.0.0-0.1.0.jar
rapids-4-spark_2.12-0.1.0.jar.1  xgboost4j-spark_3.0-1.0.0-0.1.0.jar.1
rapids-4-spark_2.12-0.1.0.jar.2  xgboost4j-spark_3.0-1.0.0-0.1.0.jar.2
rapids-4-spark_2.12-0.1.0.jar.3  xgboost4j-spark_3.0-1.0.0-0.1.0.jar.3
rapids-4-spark_2.12-0.1.0.jar.4  xgboost4j-spark_3.0-1.0.0-0.1.0.jar.4
/conten

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.14-cuda10-1.jar,/content/xgboost4j_3.0-1.0.0-0.1.0.jar,/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar,/content/rapids-4-spark_2.12-0.1.0.jar pyspark-shell'

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.plugins", "com.nvidia.spark.SQLPlugin").config("spark.rapids.memory.gpu.pooling.enabled",False).getOrCreate()
spark.sparkContext.addPyFile('/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar')
spark.sparkContext.addPyFile('/content/rapids-4-spark_2.12-0.1.0.jar')

In [ ]:
reader = spark.read

In [ ]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier

import numpy as np
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_train.parquet
!wget https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_test.parquet

--2020-08-27 04:30:50--  https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_train.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6827427 (6.5M) [application/octet-stream]
Saving to: ‘covtype_train.parquet.3’

covtype_train.parqu 100%[===================>]   6.51M  35.4MB/s    in 0.2s    

2020-08-27 04:30:50 (35.4 MB/s) - ‘covtype_train.parquet.3’ saved [6827427/6827427]

--2020-08-27 04:30:50--  https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_test.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, await

In [ ]:
!ls -alrt

total 2994600
drwxr-xr-x 13 1000 1000      4096 Jun  6 12:09 spark-3.0.0-bin-hadoop2.7
-rw-r--r--  1 root root 220272364 Jun  6 13:35 spark-3.0.0-bin-hadoop2.7.tgz.4
-rw-r--r--  1 root root 220272364 Jun  6 13:35 spark-3.0.0-bin-hadoop2.7.tgz.3
-rw-r--r--  1 root root 220272364 Jun  6 13:35 spark-3.0.0-bin-hadoop2.7.tgz.2
-rw-r--r--  1 root root 220272364 Jun  6 13:35 spark-3.0.0-bin-hadoop2.7.tgz.1
-rw-r--r--  1 root root 220272364 Jun  6 13:35 spark-3.0.0-bin-hadoop2.7.tgz
-rw-r--r--  1 root root 145993287 Jun  9 13:17 cudf-0.14-cuda10-1.jar.4
-rw-r--r--  1 root root 145993287 Jun  9 13:17 cudf-0.14-cuda10-1.jar.3
-rw-r--r--  1 root root 145993287 Jun  9 13:17 cudf-0.14-cuda10-1.jar.2
-rw-r--r--  1 root root 145993287 Jun  9 13:17 cudf-0.14-cuda10-1.jar.1
-rw-r--r--  1 root root 145993287 Jun  9 13:17 cudf-0.14-cuda10-1.jar
-rw-r--r--  1 root root   6676958 Jun 20 02:24 rapids-4-spark_2.12-0.1.0.jar.4
-rw-r--r--  1 root root   6676958 Jun 20 02:24 rapids-4-spark_2.12-0.1.0.jar.3
-rw-

In [ ]:
import pyarrow.parquet as pq

In [ ]:
pq.read_table('covtype_train.parquet')

pyarrow.Table
Elevation: double
Aspect: double
Slope: double
Horizontal_Distance_To_Hydrology: double
Vertical_Distance_To_Hydrology: double
Horizontal_Distance_To_Roadways: double
Hillshade_9am: double
Hillshade_Noon: double
Hillshade_3pm: double
Horizontal_Distance_To_Fire_Points: double
Wilderness_Area1: double
Wilderness_Area2: double
Wilderness_Area3: double
Wilderness_Area4: double
Soil_Type1: double
Soil_Type2: double
Soil_Type3: double
Soil_Type4: double
Soil_Type5: double
Soil_Type6: double
Soil_Type7: double
Soil_Type8: double
Soil_Type9: double
Soil_Type10: double
Soil_Type11: double
Soil_Type12: double
Soil_Type13: double
Soil_Type14: double
Soil_Type15: double
Soil_Type16: double
Soil_Type17: double
Soil_Type18: double
Soil_Type19: double
Soil_Type20: double
Soil_Type21: double
Soil_Type22: double
Soil_Type23: double
Soil_Type24: double
Soil_Type25: double
Soil_Type26: double
Soil_Type27: double
Soil_Type28: double
Soil_Type29: double
Soil_Type30: double
Soil_Type31: doubl

In [ ]:
train_data = reader.format('parquet').load('covtype_train.parquet')
test_data = reader.format('parquet').load('covtype_test.parquet')

In [ ]:
train_data.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_Area3|Wilder

In [ ]:
train_data.schema

StructType(List(StructField(Elevation,DoubleType,true),StructField(Aspect,DoubleType,true),StructField(Slope,DoubleType,true),StructField(Horizontal_Distance_To_Hydrology,DoubleType,true),StructField(Vertical_Distance_To_Hydrology,DoubleType,true),StructField(Horizontal_Distance_To_Roadways,DoubleType,true),StructField(Hillshade_9am,DoubleType,true),StructField(Hillshade_Noon,DoubleType,true),StructField(Hillshade_3pm,DoubleType,true),StructField(Horizontal_Distance_To_Fire_Points,DoubleType,true),StructField(Wilderness_Area1,DoubleType,true),StructField(Wilderness_Area2,DoubleType,true),StructField(Wilderness_Area3,DoubleType,true),StructField(Wilderness_Area4,DoubleType,true),StructField(Soil_Type1,DoubleType,true),StructField(Soil_Type2,DoubleType,true),StructField(Soil_Type3,DoubleType,true),StructField(Soil_Type4,DoubleType,true),StructField(Soil_Type5,DoubleType,true),StructField(Soil_Type6,DoubleType,true),StructField(Soil_Type7,DoubleType,true),StructField(Soil_Type8,DoubleType

In [ ]:
pq_file=pq.read_table('covtype_train.parquet')

In [ ]:
pq_file.column_names

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40',
 'target']

In [ ]:
label="target"
features = [ x for x in pq_file.column_names if x != label ]

In [ ]:
features

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40']

In [ ]:
import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 6, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 2
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

In [ ]:
start_time = time.time()

model=classifier.fit(train_data)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

GPU Training Time: 16.036097288131714 seconds


In [ ]:
!nvidia-smi

Thu Aug 27 04:36:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    34W /  70W |   1319MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model.write().overwrite().save('/content/model/')

In [ ]:
!ls

covtype_test.parquet		 rapids-4-spark_2.12-0.1.0.jar.4
covtype_test.parquet.1		 sample_data
covtype_test.parquet.2		 spark-3.0.0-bin-hadoop2.7
covtype_test.parquet.3		 spark-3.0.0-bin-hadoop2.7.tgz
covtype_train.parquet		 spark-3.0.0-bin-hadoop2.7.tgz.1
covtype_train.parquet.1		 spark-3.0.0-bin-hadoop2.7.tgz.2
covtype_train.parquet.2		 spark-3.0.0-bin-hadoop2.7.tgz.3
covtype_train.parquet.3		 spark-3.0.0-bin-hadoop2.7.tgz.4
cudf-0.14-cuda10-1.jar		 xgboost4j_3.0-1.0.0-0.1.0.jar
cudf-0.14-cuda10-1.jar.1	 xgboost4j_3.0-1.0.0-0.1.0.jar.1
cudf-0.14-cuda10-1.jar.2	 xgboost4j_3.0-1.0.0-0.1.0.jar.2
cudf-0.14-cuda10-1.jar.3	 xgboost4j_3.0-1.0.0-0.1.0.jar.3
cudf-0.14-cuda10-1.jar.4	 xgboost4j_3.0-1.0.0-0.1.0.jar.4
model				 xgboost4j-spark_3.0-1.0.0-0.1.0.jar
rapids-4-spark_2.12-0.1.0.jar	 xgboost4j-spark_3.0-1.0.0-0.1.0.jar.1
rapids-4-spark_2.12-0.1.0.jar.1  xgboost4j-spark_3.0-1.0.0-0.1.0.jar.2
rapids-4-spark_2.12-0.1.0.jar.2  xgboost4j-spark_3.0-1.0.0-0.1.0.jar.3
rapids-4-spark_2.12-0.1.0.ja

In [ ]:
loaded_model = XGBoostClassificationModel().load('/content/model/')

In [ ]:
result=loaded_model.transform(test_data)

In [ ]:
result.select(label, 'rawPrediction', 'probability', 'prediction').show(5)

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|     1|[-0.7823809385299...|[0.21761906147003...|       1.0|
|     4|[-3.2074680328369...|[-2.2074680328369...|       1.0|
|     4|[-3.1274266242980...|[-2.1274266242980...|       1.0|
|     4|[-3.3601572513580...|[-2.3601572513580...|       1.0|
|     4|[-2.9516975879669...|[-1.9516975879669...|       1.0|
+------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result)